In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

DB_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine_entrada = create_engine(DB_URL)


query = "SELECT * FROM Analista"


dados_originais_df = pd.read_sql(query, engine_entrada)


dados_originais_df = dados_originais_df.copy()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found

In [ ]:
# pseudonimizar nome
def pseudonimizar_nome(nome):
    # percorre o tamanho do nome recebido (cada letra)
    for i in range(len(nome)):
        # cria uma string com o texto "Analista" e o número da posição
        nome = str("Analista " + str(i))
        # retorna o nome (mas aqui ele para logo na primeira volta do loop)
        return nome
# copia os dados originais para um novo dataframe
dados_anonimizados = dados_originais_df.copy()
#Faz com que o nome do analista seja impresso como "Analista 1", "Analista 2", etc
dados_anonimizados["nome_pseudonimizado"] = [
    f"Analista {i+1}" for i in range(len(dados_anonimizados))
]
#Imprime a anonimização 
print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["nome_pseudonimizado"]])


==== Dados anonimizados ====


,nome_pseudonimizado
0,Analista 1
1,Analista 2
2,Analista 3
3,Analista 4
4,Analista 5
5,Analista 6
6,Analista 7
7,Analista 8
8,Analista 9
9,Analista 10


In [30]:
import bcrypt

def criptografar(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))
dados_anonimizados = dados_originais_df.copy()

dados_anonimizados["senha_anonimizada"] = dados_anonimizados["senha"].apply(criptografar)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["senha_anonimizada"]])





==== Dados anonimizados ====


,senha_anonimizada
0,$2b$12$t1cKLkvIxp4kTUNc.yhmAemZho6ZC.kapKJKHSj...
1,$2b$12$wef6325d4RKxwTpijcP/sO4fGPXivUjlgkYAYDv...
2,$2b$12$e1RDxe95CIhCBBZ0mGAcT.cvLO5r1O26ziMNyJJ...
3,$2b$12$dJfJ1Uuj5SFOdbhnMUPBGeRqxXRNnqlZFYe5O5i...
4,$2b$12$Cf9uusgb88N8MHiM.q3ZEOZr17OkzVWCiEw/PGj...
5,$2b$12$WyEJgxhjA0mDORaDVYYICu7d5ZZdwKbMYd3.kIV...
6,$2b$12$84WzZdJjnhEGmhaKrJNSvuFj7MPVdmTxgy74tLn...
7,$2b$12$IVpFPlZnqNHGZf..ZI017uPC.bQGgCutPW5vE.f...
8,$2b$12$1TPuiAJcWux21utYIt9cDON4S9X.u9lSCUrLOAE...
9,$2b$12$PiyqqcS3H2MrG2hU58jNCOQ94EjDX9SgzsaETzt...


In [ ]:
# Pseudonimização da data de contratação
from IPython.display import display


def pseudonimizar_data(dt_contratacao):
   # Garante que é string para que possa remover espaços 
    data = str(dt_contratacao).strip()
    
    # Remove hífen e barra
    data = data.replace("-", "").replace("/", "")
    
    # Função para anonimizar 4 primeiros caracteres
    if len(data) > 4:
        return data[:4] + "****"
    else:
        return data + "****"
dados_anonimizados = dados_originais_df.copy()

# Aplica e mostra a pseudonimização
dados_anonimizados["data_contratacao_pseudo"] = dados_anonimizados["dt_contratacao"].apply(pseudonimizar_data)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["dt_contratacao", "data_contratacao_pseudo"]])


==== Dados anonimizados ====


,dt_contratacao,data_contratacao_pseudo
0,2023-01-15,2023****
1,2023-02-20,2023****
2,2023-03-10,2023****
3,2023-04-05,2023****
4,2023-05-12,2023****
5,2023-06-18,2023****
6,2023-07-22,2023****
7,2023-08-30,2023****
8,2023-09-15,2023****
9,2023-10-08,2023****


In [ ]:
#Pseudonimização cargo
from IPython.display import display

def pseudonimizar(cargo):
    # Garante que é string para que possa remover espaços 
    palavra = str(cargo).strip()
    
    # Remove hífens e espaços
    palavra = palavra.replace("-", "").replace(" ", "")
    
    # Substitui a partir do 5º caractere por ****
    if len(palavra) > 3:
        valor_pseudo = palavra[:3] + "****"
    else:
        valor_pseudo = palavra + "****"
    
    return valor_pseudo

# Cria cópia do DataFrame original (opcional, pra não alterar os dados reais)
dados_anonimizados = dados_originais_df.copy()

# Aplica pseudonimização na coluna "cargo"
dados_anonimizados["cargo_pseudonimizado"] = dados_anonimizados["cargo"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["cargo_pseudonimizado"]])



==== Dados anonimizados ====


,cargo_pseudonimizado
0,Ana****
1,Ana****
2,Ana****
3,Coo****
4,Ana****
5,Ana****
6,Ana****
7,Ana****
8,Sup****
9,Ana****


In [6]:
# Anonimização analista 

from IPython.display import display


def anonimizar(senha, chave):
    senha = str(senha)
    chave = str(chave).lower()
    senha_cripto = ''
    i = 0
    for l in senha:
        deslocamento = ord(chave[i % (len(chave))]) - ord('a')
        nova_letra = chr((ord(l) - 32 + deslocamento) % 95 + 32)
        senha_cripto += nova_letra
        i += 1
    return senha_cripto

# cópia
dados_anonimizados = dados_originais_df.copy()


chave = "minhachave"
dados_anonimizados["cpf_anonimizado"] = dados_anonimizados["cpf"].apply(lambda x: anonimizar(x, chave))

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["cpf_anonimizado"]])



==== Dados anonimizados ====


,cpf_anonimizado
0,=:@;58>8N5>
1,>;A<69?9E6?
2,?<B=7:@0F7@
3,@=C>8;71G8A
4,A>D?9282H9B
5,B?E@0393I:C
6,C@F714:4J;D
7,DA=825;5K<E
8,E8>936<6L==
9,<9?:47=7M=>


In [7]:
# Pseudonimização email
def pseudonimizar(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:2] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio
dados_anonimizados = dados_originais_df.copy()
dados_anonimizados["email_pseudonimizado"] = dados_anonimizados["email"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["email_pseudonimizado"]])


==== Dados anonimizados ====


,email_pseudonimizado
0,ra****@eitruck.com
1,ca****@eitruck.com
2,le****@eitruck.com
3,fe****@eitruck.com
4,ga****@eitruck.com
5,am****@eitruck.com
6,th****@eitruck.com
7,la****@eitruck.com
8,fe****@eitruck.com
9,na****@eitruck.com
